**Lesson 4**

**How to Run the Game**
- Whenever you want to run the game, make sure to first run the import block, then the block with the spaceship class, followed by whatever else you want to run for implementing the shooting mechanics. Finally run the code block labelled 'Main'

In [ ]:
import pygame
import random
import math


# Initialize PyGame
pygame.init()

# Screen dimensions
WIDTH, HEIGHT = 800, 600
Y_PADDING = 5
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Space Shooter")

# Colours
BLACK = (0, 0, 0)

# Load spaceship image
SPACESHIP_IMAGE = pygame.image.load("spaceship.png")
SPACESHIP_IMAGE = pygame.transform.scale(SPACESHIP_IMAGE, (50, 50))  # Resize to fit

# Load bullet image
BULLET_IMAGE = pygame.image.load("bullet.png")  # Ensure bullet.png is in the same directory
BULLET_IMAGE = pygame.transform.scale(BULLET_IMAGE, (10, 20))  # Resize for appropriate scaling

# Load asteroid image
ASTEROID_IMAGE = pygame.image.load("asteroid.png")  # Ensure asteroid.png is in the same directory
ASTEROID_IMAGE = pygame.transform.scale(ASTEROID_IMAGE, (40, 40))  # Resize for appropriate scaling

**Spaceship Class**

_Replace below code with your spaceship class if you completed the 4th movement implementation. If not then use the code provided._


In [ ]:
### Replace this class with your implementation 4 ###

class Spaceship:
    def __init__(self, x, y):
        self.__x = x
        self.__y = y
        self.__image = SPACESHIP_IMAGE
        self.__width = self.__image.get_width()
        self.__height = self.__image.get_height()
        self.__velocity = 3
        self.__angle = 0

    # Getters
    def get_x(self):
        return self.__x

    def get_y(self):
        return self.__y

    def get_velocity(self):
        return self.__velocity

    def get_width(self):
        return self.__width

    def get_height(self):
        return self.__height

    def get_angle(self):
        return self.__angle

    # Setters
    def set_x(self, x):
        self.__x = x

    def set_y(self, y):
        self.__y = y

    def set_velocity(self, velocity):
        self.__velocity = velocity

    def set_angle(self, angle):
        self.__angle = angle

    # Draw method
    def draw(self, screen):
        rotated_image = pygame.transform.rotate(self.__image, self.__angle)
        new_rect = rotated_image.get_rect(center=self.__image.get_rect(topleft=(self.__x, self.__y - 50)).center)
        screen.blit(rotated_image, new_rect.topleft)

    # Move method with internal usage of getters and setters
    def move(self, keys):
        dx, dy = 0, 0  # Change in x and y
        angle = 0  # Default angle

        if keys[pygame.K_LEFT] and self.__x > 0:
            dx = -self.__velocity
            angle = 90
        if keys[pygame.K_RIGHT] and self.__x + self.__width < WIDTH:
            dx = self.__velocity
            angle = -90
        if keys[pygame.K_UP] and self.__y - self.__height > 0:
            dy = -self.__velocity
            angle = 0
        if keys[pygame.K_DOWN] and self.__y + Y_PADDING <= HEIGHT:
            dy = self.__velocity
            angle = 180

        # Handle diagonal movement
        if dx < 0 and dy < 0:
            angle = 45  # Up-left
        elif dx > 0 and dy < 0:
            angle = -45  # Up-right
        elif dx < 0 and dy > 0:
            angle = 135  # Down-left
        elif dx > 0 and dy > 0:
            angle = -135  # Down-right

        # Apply movement
        self.__x += dx
        self.__y += dy
        self.__angle = angle  # Update rotation angle

    def get_rect(self):
        return pygame.Rect(self.__x, self.__y, self.__width, self.__height)


**1. Asteroid Class**
- Now that you have a spaceship that can shoot bullets, we need the asteroids to actually make this a game that's worth playing.
- This astroid class needs to:
    - Spawn asteroids at regular intervals on the edge of the window (can spawn from any of the 4 sides)
    - The astroids should travel towards the centre (towards the spaceship)


In [ ]:
class Asteroid:
    def __init__(self):
        # Randomly choose to spawn on one of the four edges of the screen
        edge = random.choice(["top", "bottom", "left", "right"])

        if edge == "top":
            self.x = random.randint(0, WIDTH)
            self.y = 0  # Top edge
        elif edge == "bottom":
            self.x = random.randint(0, WIDTH)
            self.y = HEIGHT  # Bottom edge
        elif edge == "left":
            self.x = 0  # Left edge
            self.y = random.randint(0, HEIGHT)
        elif edge == "right":
            self.x = WIDTH  # Right edge
            self.y = random.randint(0, HEIGHT)

        # Calculate direction vector (dx, dy) to the center of the screen
        center_x, center_y = WIDTH // 2, HEIGHT // 2
        distance = math.sqrt((center_x - self.x) ** 2 + (center_y - self.y) ** 2)
        self.dx = (center_x - self.x) / distance
        self.dy = (center_y - self.y) / distance

        # Set velocity
        self.velocity = random.uniform(2, 5)  # Random speed between 2 and 5

        # Use the asteroid image's dimensions for size
        self.width = ASTEROID_IMAGE.get_width()
        self.height = ASTEROID_IMAGE.get_height()

    def move(self):
        # Update position based on velocity and movement vector
        self.x += self.dx * self.velocity
        self.y += self.dy * self.velocity

    def draw(self, screen):
        # Draw the asteroid using the asteroid image
        screen.blit(ASTEROID_IMAGE, (self.x - self.width // 2, self.y - self.height // 2))  # Centering the image

    def get_rect(self):
        return pygame.Rect(self.x, self.y, self.width, self.height)


**2. Work on the Bullet Class from Last Lesson**
- Now that you have asteroids that spawn. You can work on the bullet class implementations below
- _Copy the code you have from last lesson and continue working here_

**2a. Bullet Class**
- You will now need to implement the bullet class. I have given you the skeleton for the class but you will need to think about what attributes are required
    - The bullets need to fire from the centre of the spaceship
    - They should travel upwards
    - You need to press spacebar to shoot the bullets, holding the spacebar will continiously shoot the bullet


*Things to think about*
- All the attributes you will need for the bullet class, what sort of data would you need about a bullet?
- Are all the attributes private? Have you created the getters and setters?
- When would you need to make a new bullet
- How will you keep track of all the bullets?
- What do you do once the bullet is off the screen
- Holding down the space bar will make 60 bullets a second (probably not what you want). Figure out how to delay each shot so that it's not just a streak of bullets

*Rough order of implementation*
- Complete the Bullet class and complete all the methods
- Then modify main to implement the change

In [ ]:
# Bullet class
class Bullet:
    def __init__(self, x, y):
        self.__x = x  # Private x-coordinate
        self.__y = y  # Private y-coordinate
        self.__velocity = 7  # Speed of the bullet
        self.__image = BULLET_IMAGE  # Use the provided BULLET_IMAGE
        self.__width = self.__image.get_width()
        self.__height = self.__image.get_height()

    # Getters
    def get_x(self):
        return self.__x

    def get_y(self):
        return self.__y

    def get_velocity(self):
        return self.__velocity

    def get_width(self):
        return self.__width

    def get_height(self):
        return self.__height

    # Setters
    def set_x(self, x):
        self.__x = x

    def set_y(self, y):
        self.__y = y

    def set_velocity(self, velocity):
        self.__velocity = velocity

    # Draw method
    def draw(self, screen):
        screen.blit(self.__image, (self.__x, self.__y))

    # Move method
    def move(self):
        self.__y -= self.__velocity

**2b. Improve Bullet Class**
- Now that your bullet class is working, change it so that the bullet fires in the direction that it is facing. For example, if the spaceship is facing the top left, then the bullet also needs to shoot in that direction.
- Feel free to make any other improvements that you can think of.
    - How can you make it shoot 3 bullets instead of 1?


*Prerequisite*
- In order to implement this, your spaceship will need to face in the direction it is travelling. If you are using my implementation then this is already done for you, however, if you are using your own implementation, please make sure that this works.

In [ ]:
class Bullet:
    def __init__(self, x, y, angle):
        self.__x = x  # Private x-coordinate
        self.__y = y  # Private y-coordinate
        self.__velocity = 7  # Fixed speed for all directions
        self.__image = BULLET_IMAGE
        self.__width = self.__image.get_width()
        self.__height = self.__image.get_height()
        self.__angle = angle  # Bullet angle

        # Movement logic (fixed directional movement)
        if self.__angle == 0:    # Up
            self.__dx, self.__dy = 0, -self.__velocity
        elif self.__angle == 90:   # Left
            self.__dx, self.__dy = -self.__velocity, 0
        elif self.__angle == -90:  # Right
            self.__dx, self.__dy = self.__velocity, 0
        elif self.__angle == 180:  # Down
            self.__dx, self.__dy = 0, self.__velocity
        elif self.__angle == 45:   # Up-Left
            self.__dx, self.__dy = -self.__velocity, -self.__velocity
        elif self.__angle == -45:  # Up-Right
            self.__dx, self.__dy = self.__velocity, -self.__velocity
        elif self.__angle == 135:  # Down-Left
            self.__dx, self.__dy = -self.__velocity, self.__velocity
        elif self.__angle == -135: # Down-Right
            self.__dx, self.__dy = self.__velocity, self.__velocity
        else:
            self.__dx, self.__dy = 0, -self.__velocity  # Default to moving up

    # Getters
    def get_x(self):
        return self.__x

    def get_y(self):
        return self.__y

    def get_velocity(self):
        return self.__velocity

    def get_width(self):
        return self.__width

    def get_height(self):
        return self.__height

    def get_angle(self):
        return self.__angle

    def set_velocity(self, velocity):
        self.__velocity = velocity

    # Draw method (Ensures bullet image rotates properly)
    def draw(self, screen):
        rotated_image = pygame.transform.rotate(self.__image, self.__angle)
        new_rect = rotated_image.get_rect(center=self.__image.get_rect(topleft=(self.__x, self.__y - 20)).center)
        screen.blit(rotated_image, new_rect.topleft)

    # Move method
    def move(self):
        self.__x += self.__dx  # Move in the x direction
        self.__y += self.__dy  # Move in the y direction


**2c. Fast Bullet Implementation**
- Create a new bullet class called `FastBullet` where the bullet travels faster than the normal bullet.
- One key difference with this bullet is that it will only be able to travel 100 pixels from it's starting position before it disappears.

*How to implement it*
- Make this bullet only fire when the `F` key is pressed (you can change it and make it any other key if you like)
- This should not change how the normal bullet works, that should still fire when you press the space bar.


In [ ]:
# Fast Bullet class
class FastBullet:
    def __init__(self, x, y, angle):
        self.__x = x  # Private x-coordinate
        self.__y = y  # Private y-coordinate
        self.__velocity = 12  # Faster speed compared to normal bullets
        self.__image = BULLET_IMAGE
        self.__width = self.__image.get_width()
        self.__height = self.__image.get_height()
        self.__angle = angle  # Bullet angle
        self.__starting_y = y
        self.__starting_x = x
        self.__max_pixel_travel = 250

        # Movement logic (based on angle)
        if self.__angle == 0:    # Up
            self.__dx, self.__dy = 0, -self.__velocity
        elif self.__angle == 90:   # Left
            self.__dx, self.__dy = -self.__velocity, 0
        elif self.__angle == -90:  # Right
            self.__dx, self.__dy = self.__velocity, 0
        elif self.__angle == 180:  # Down
            self.__dx, self.__dy = 0, self.__velocity
        elif self.__angle == 45:   # Up-Left
            self.__dx, self.__dy = -self.__velocity, -self.__velocity
        elif self.__angle == -45:  # Up-Right
            self.__dx, self.__dy = self.__velocity, -self.__velocity
        elif self.__angle == 135:  # Down-Left
            self.__dx, self.__dy = -self.__velocity, self.__velocity
        elif self.__angle == -135: # Down-Right
            self.__dx, self.__dy = self.__velocity, self.__velocity
        else:
            self.__dx, self.__dy = 0, -self.__velocity  # Default to moving up

    # Getters
    def get_x(self):
        return self.__x

    def get_y(self):
        return self.__y

    def get_velocity(self):
        return self.__velocity

    def get_width(self):
        return self.__width

    def get_height(self):
        return self.__height

    def get_angle(self):
        return self.__angle

    def set_velocity(self, velocity):
        self.__velocity = velocity

    # Draw method (Ensures bullet image rotates properly)
    def draw(self, screen):
        rotated_image = pygame.transform.rotate(self.__image, self.__angle)
        new_rect = rotated_image.get_rect(center=self.__image.get_rect(topleft=(self.__x, self.__y - 20)).center)
        screen.blit(rotated_image, new_rect.topleft)

    # Move method (ensuring bullet disappears after 100 pixels)
    def move(self):
        self.__x += self.__dx  # Move in the x direction
        self.__y += self.__dy  # Move in the y direction

        # Check if bullet has traveled more than max_pixel_travel
        if abs(self.__y - self.__starting_y) > self.__max_pixel_travel or abs(self.__x - self.__starting_x) > self.__max_pixel_travel:
            return False  # Indicates the bullet should be removed
        return True

**2d. Ricochet Bullet**
- This is an extension so only do this once you have completed everything before this point!
- Implement a bullet that can bounce off 2 walls before disheartening

*Things to think about*
- How will you keep track of how many times it's bounced?
- Think about the image and what direction it is facing (look at how it's done for the spaceship class to give you an idea)

*How to implement it*
- Make this bullet only fire when the `R` key is pressed (you can change it and make it any other key if you like)
- This should not change how the normal bullet works, that should still fire when you press the space bar.


In [ ]:
class RicochetBullet:
    def __init__(self, x, y, angle):
        self.__x = x
        self.__y = y
        self.__velocity = 5
        self.__image = BULLET_IMAGE
        self.__angle = angle
        self.__width = self.__image.get_width()
        self.__height = self.__image.get_height()
        self.__bounces = 0
        self.__max_bounces = 2

        # Movement logic (fixed directional movement)
        self.__dx, self.__dy = self.__get_direction_from_angle()

    def __get_direction_from_angle(self):
        if self.__angle == 0:    # Up
            return 0, -self.__velocity
        elif self.__angle == 90:   # Left
            return -self.__velocity, 0
        elif self.__angle == -90:  # Right
            return self.__velocity, 0
        elif self.__angle == 180:  # Down
            return 0, self.__velocity
        elif self.__angle == 45:   # Up-Left
            return -self.__velocity, -self.__velocity
        elif self.__angle == -45:  # Up-Right
            return self.__velocity, -self.__velocity
        elif self.__angle == 135:  # Down-Left
            return -self.__velocity, self.__velocity
        elif self.__angle == -135: # Down-Right
            return self.__velocity, self.__velocity
        else:
            return 0, -self.__velocity  # Default to moving up

    def draw(self, screen):
        rotated_image = pygame.transform.rotate(self.__image, self.__angle)
        new_rect = rotated_image.get_rect(center=self.__image.get_rect(topleft=(self.__x, self.__y - 20)).center)
        screen.blit(rotated_image, new_rect.topleft)

    def move(self):
        self.__x += self.__dx
        self.__y += self.__dy

        # Check for collisions with walls and bounce
        if self.__x < 0 or self.__x + self.__width > WIDTH:  # Collides with left or right wall
            self.__dx = -self.__dx  # Reverse x-direction
            self.__angle = 180 - self.__angle  # Correct angle flipping for diagonals
            self.__bounces += 1

        if self.__y - self.__height< 0 or self.__y > HEIGHT:  # Collides with top or bottom wall
            self.__dy = -self.__dy  # Reverse y-direction
            if self.__angle == 180:
                self.__angle = 0
            elif self.__angle == 0:
                self.__angle = 180
            else:
                self.__angle = -self.__angle  # Correct angle flipping for diagonals
            self.__bounces += 1


        # Return False if the bullet has bounced too many times
        return self.__bounces <= self.__max_bounces

    # Getters
    def get_x(self):
        return self.__x

    def get_y(self):
        return self.__y

    def get_velocity(self):
        return self.__velocity

    def get_bounces(self):
        return self.__bounces

    def get_angle(self):
        return self.__angle


**MAIN**

In [ ]:
# Main game loop
def main():
    clock = pygame.time.Clock()
    run = True
    spaceship = Spaceship(WIDTH // 2, HEIGHT - 60)
    bullets = []  # List to store bullets
    last_bullet_time = 0  # Time when the last normal bullet was fired
    bullet_cooldown = 100  # Cooldown time in milliseconds for normal bullets

    fast_bullets = []  # List to store fast bullets
    last_fast_bullet_time = 0  # Time when the last fast bullet was fired
    fast_bullet_cooldown = 300  # Cooldown time in milliseconds for fast bullets

    ricochet_bullets = []  # List to store ricochet bullets
    last_ricochet_bullet_time = 0  # Time when the last ricochet bullet was fired
    ricochet_bullet_cooldown = 500  # Cooldown time in milliseconds for ricochet bullets

    asteroids = []  # List to track active asteroids
    asteroid_spawn_timer = 0  # Timer to control asteroid spawn interval

    while run:
        clock.tick(60)
        screen.fill(BLACK)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

        keys = pygame.key.get_pressed()
        spaceship.move(keys)

        # Handle shooting normal bullets with cooldown
        current_time = pygame.time.get_ticks()
        if keys[pygame.K_SPACE] and current_time - last_bullet_time > bullet_cooldown:
            # Create a new bullet at the spaceship's center position
            bullet = Bullet(
                x=spaceship.get_x() + spaceship.get_width() // 2 - BULLET_IMAGE.get_width() // 2,
                y=spaceship.get_y() - BULLET_IMAGE.get_height(),
                angle=spaceship.get_angle()
            )
            bullets.append(bullet)
            last_bullet_time = current_time  # Update the last bullet time

        # Handle shooting fast bullets with cooldown
        if keys[pygame.K_f] and current_time - last_fast_bullet_time > fast_bullet_cooldown:
            # Create a new fast bullet at the spaceship's center position
            fast_bullet = FastBullet(
                x=spaceship.get_x() + spaceship.get_width() // 2 - BULLET_IMAGE.get_width() // 2,
                y=spaceship.get_y() - BULLET_IMAGE.get_height(),
                angle=spaceship.get_angle()
            )
            fast_bullets.append(fast_bullet)
            last_fast_bullet_time = current_time  # Update the last fast bullet time

        # Handle shooting ricochet bullets with cooldown
        if keys[pygame.K_r] and current_time - last_ricochet_bullet_time > ricochet_bullet_cooldown:
            # Create a new ricochet bullet at the spaceship's center position
            ricochet_bullet = RicochetBullet(
                x=spaceship.get_x() + spaceship.get_width() // 2 - BULLET_IMAGE.get_width() // 2,
                y=spaceship.get_y() - BULLET_IMAGE.get_height(),
                angle=spaceship.get_angle()
            )
            ricochet_bullets.append(ricochet_bullet)
            last_ricochet_bullet_time = current_time  # Update the last ricochet bullet time

        # Update and draw normal bullets
        for bullet in bullets[:]:
            bullet.move()
            bullet.draw(screen)
            # Remove bullets that go off-screen
            if bullet.get_y() + bullet.get_height() < 0 or bullet.get_x() < 0 or bullet.get_x() > WIDTH:
                bullets.remove(bullet)

        # Update and draw fast bullets
        for fast_bullet in fast_bullets[:]:
            if not fast_bullet.move():  # Move and check if it should still exist
                fast_bullets.remove(fast_bullet)
            else:
                fast_bullet.draw(screen)

        # Update and draw ricochet bullets
        for ricochet_bullet in ricochet_bullets[:]:
            if not ricochet_bullet.move():  # Move and check if it should still exist
                ricochet_bullets.remove(ricochet_bullet)
            else:
                ricochet_bullet.draw(screen)

        # Asteroid spawning logic
        asteroid_spawn_timer += 1
        if asteroid_spawn_timer >= 60:  # Spawn an asteroid every 60 frames (1 second at 60 FPS)
            asteroids.append(Asteroid())  # Add a new asteroid to the list
            asteroid_spawn_timer = 0

        # Move and draw all active asteroids
        for asteroid in asteroids:
            asteroid.move()
            asteroid.draw(screen)

            # Collision detection between asteroid and spaceship
            if asteroid.get_rect().colliderect(spaceship.get_rect()):
                print("Collision detected! Game Over!")
                run = False

        spaceship.draw(screen)
        pygame.display.flip()

    pygame.quit()

if __name__ == "__main__":
    main()